# Nominal amplitude model

```{autolink-concat}
```

In [ ]:
import logging
import os

import sympy as sp
from IPython.display import Markdown, display

from polarimetry.amplitude import simplify_latex_rendering
from polarimetry.io import as_markdown_table, display_latex, perform_cached_doit
from polarimetry.lhcb import load_model_builder, load_model_parameters
from polarimetry.lhcb.particle import K, Λc, Σ, load_particles, p, π

simplify_latex_rendering()

NO_TQDM = "EXECUTE_NB" in os.environ
if NO_TQDM:
    logging.getLogger().setLevel(logging.ERROR)
    logging.getLogger("polarimetry.io").setLevel(logging.ERROR)

## Resonances and LS-scheme

Particle definitions for $\Lambda_c^+$ and $p, \pi^+, K^-$ in the sequential order.

In [ ]:
decay_particles = [Λc, p, π, K, Σ]
Markdown(as_markdown_table(decay_particles))

Particle definitions as defined in {download}`particle-definitions.yaml <../data/particle-definitions.yaml>`:

In [ ]:
particles = load_particles("../data/particle-definitions.yaml")
resonances = [p for p in particles.values() if p not in set(decay_particles)]
src = as_markdown_table(resonances)
Markdown(src)

## Amplitude

### Spin-alignment amplitude

The full intensity of the amplitude model is obtained by summing the following aligned amplitude over all helicity values $\lambda_i$ in the initial state $0$ and final states $1, 2, 3$:

In [ ]:
model_choice = 0
amplitude_builder = load_model_builder(
    model_file="../data/model-definitions.yaml",
    particle_definitions=particles,
    model_id=model_choice,
)
model = amplitude_builder.formulate()

In [ ]:
def simplify_notation(expr):
    def substitute_node(node):
        if isinstance(node, sp.Indexed):
            if node.indices[2:] == (0, 0):
                return sp.Indexed(node.base, *node.indices[:2])
        return node

    for node in sp.preorder_traversal(expr):
        new_node = substitute_node(node)
        expr = expr.xreplace({node: new_node})
    return expr


display(simplify_notation(model.intensity.args[0].args[0].args[0].cleanup()))

Note that we simplified notation here: the amplitude indices for the spinless states are not rendered and their corresponding Wigner-$d$ alignment functions are simply $1$.

The relevant $\zeta^i_{j(k)}$ angles are {doc}`defined as</appendix/angles>`:

In [ ]:
display_latex({k: v for k, v in model.variables.items() if "zeta" in str(k)})

### Sub-system amplitudes

In [ ]:
display_latex({simplify_notation(k): v for k, v in model.amplitudes.items()})

The $\theta_{ij}$ angles are {doc}`defined as</appendix/angles>`:

In [ ]:
display_latex({k: v for k, v in model.variables.items() if "theta" in str(k)})

Definitions for the $\phi_{ij}$ angles can be found under {doc}`/appendix/angles`.

## Parameter definitions

Parameter values are provided in {download}`model-definitions.yaml<../data/model-definitions.yaml>`, but the **keys** of the helicity couplings have to remapped to the helicity **symbols** that are used in this amplitude model. The function {func}`.parameter_key_to_symbol` implements this remapping, following the [supplementary material](https://cds.cern.ch/record/2824328/files) of [DOI:10.48550/arXiv.2208.03262](https://doi.org/10.48550/arXiv.2208.03262). It is asserted below that:
1. the keys are mapped to symbols that exist in the nominal amplitude model
2. all parameter symbols in the nominal amplitude model have a value assigned to them.

In [ ]:
imported_parameter_values = load_model_parameters(
    "../data/model-definitions.yaml", amplitude_builder.decay, model_choice
)
unfolded_intensity_expr = perform_cached_doit(model.full_expression)
model_symbols = unfolded_intensity_expr.free_symbols

non_existent = set(imported_parameter_values) - set(model_symbols)
error_message = "Imported symbols that don't exist in model:\n  "
error_message += "\n  ".join(map(str, sorted(non_existent, key=str)))
assert non_existent == set(), error_message

undefined = (
    set(model_symbols)
    - set(imported_parameter_values)
    - set(model.parameter_defaults)
    - set(model.variables)
    - set(sp.symbols("sigma1:4", nonnegative=True))
)
undefined = {
    s
    for s in undefined
    if not str(s).endswith("{decay}")
    if not str(s).endswith("production}")
}
error_message = "Symbols in model that don't have a definition:\n  "
error_message += "\n  ".join(map(str, sorted(undefined, key=str)))
assert undefined == set(), error_message
model.parameter_defaults.update(imported_parameter_values)

### Helicity coupling values

#### Production couplings

In [ ]:
production_couplings = {
    key: value
    for key, value in model.parameter_defaults.items()
    if isinstance(key, sp.Indexed)
    if "production" in str(key.base)
    if str(value) != "1"
}
display_latex(production_couplings)

#### Decay couplings

In [ ]:
decay_couplings = {
    key: value
    for key, value in model.parameter_defaults.items()
    if isinstance(key, sp.Indexed)
    if "decay" in str(key.base)
}
display_latex(decay_couplings)

### Non-coupling parameters

In [ ]:
couplings = set(production_couplings) | set(decay_couplings)
non_coupling_parameters = {
    symbol: model.parameter_defaults[symbol]
    for symbol in sorted(model.parameter_defaults, key=str)
    if not isinstance(symbol, sp.Indexed)
}
display_latex(non_coupling_parameters)